In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.3/905.3 kB 20.0 MB/s eta 0:00:00


In [2]:
import torch
import os
import glob
from PIL import Image
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Example usage:
BKAI_train_images_folder = '/kaggle/input/bkai-dataset2/vietnamese/train_images'
BKAI_labels_folder = '/kaggle/input/bkai-dataset2/vietnamese/labels'

BKAI_val_images_folder = '/kaggle/input/bkai-dataset2/vietnamese/test_image'

BKAI_test_images_folder = '/kaggle/input/bkai-dataset2/vietnamese/unseen_test_images'

BKAI_train_output_images_folder = "/kaggle/working/train/images"
BKAI_train_output_labels_folder = "/kaggle/working/train/labels"

BKAI_val_output_images_folder = "/kaggle/working/val/images"
BKAI_val_output_labels_folder = "/kaggle/working/val/labels"

BKAI_test_output_images_folder = "/kaggle/working/test/images"
BKAI_test_output_labels_folder = "/kaggle/working/test/labels"

# Define paths
BKAI1_train_images_folder = '/kaggle/input/bkai-dataset-1/train_imgs/training_img'
BKAI1_train_labels_folder = '/kaggle/input/bkai-dataset-1/train_gt/training_gt'
BKAI1_train_output_images_folder = "/kaggle/working/train/images"
BKAI1_train_output_labels_folder = "/kaggle/working/train/labels"

In [4]:
# Total


total_test_images_folder = "/kaggle/input/totaltextstr/Total-Text/Test"
total_test_labels_folder = "/kaggle/input/totaltextstr/Total-Text/Annotation/groundtruth_polygonal_annotation/Test"

total_test_output_images_folder = "/kaggle/working/test_total/images"
total_test_output_labels_folder = "/kaggle/working/test_total/labels"

In [5]:
# ICDAR2015
icdar2015_test_images_folder = "/kaggle/input/icdar2015/ch4_test_images"
icdar2015_test_labels_folder = "/kaggle/input/icdar2015/ch4_test_localization_transcription_gt"

icdar2015_test_output_images_folder = "/kaggle/working/test_icdar2015/images"
icdar2015_test_output_labels_folder = "/kaggle/working/test_icdar2015/labels"

# Icdar2015 Dataset Test


In [6]:
def Icidar2015_process_annotations(train_images_folder, train_labels_folder, train_output_images_folder, train_output_labels_folder):
    # Tạo các thư mục đầu ra nếu chưa tồn tại
    os.makedirs(train_output_images_folder, exist_ok=True)
    os.makedirs(train_output_labels_folder, exist_ok=True)

    # Hàm chuyển đổi tọa độ polygon sang YOLO bounding box
    def polygon_to_yolo_bbox(x, y, width_img, height_img):
        x_min, x_max = min(x), max(x)
        y_min, y_max = min(y), max(y)
        x_center = (x_min + x_max) / 2 / width_img
        y_center = (y_min + y_max) / 2 / height_img
        bbox_width = (x_max - x_min) / width_img
        bbox_height = (y_max - y_min) / height_img
        return x_center, y_center, bbox_width, bbox_height

    # Duyệt qua các file annotation
    annotation_files = glob.glob(os.path.join(train_labels_folder, "*.txt"))

    for anno_file in annotation_files:
        base_name = os.path.basename(anno_file).replace("gt_", "").replace(".txt", "")
        image_path = os.path.join(train_images_folder, f"{base_name}.jpg")
        
        try:
            # Mở ảnh để lấy kích thước
            img = Image.open(image_path)
            width_img, height_img = img.size

            # Lưu ảnh vào thư mục đầu ra
            output_image_path = os.path.join(train_output_images_folder, f"{base_name}.jpg")
            
            # Biến kiểm tra xem có bounding box hợp lệ không
            valid_boxes = False
            
            # Lưu file annotation mới vào thư mục đầu ra
            output_label_file = os.path.join(train_output_labels_folder, f"{base_name}.txt")
            
            with open(anno_file, "r", encoding="utf-8-sig") as f, open(output_label_file, "w") as out_f:
                for line in f:
                    try:
                        # Loại bỏ ký tự BOM nếu có
                        line = line.lstrip('\ufeff')

                        # Tách các phần tử trong dòng annotation
                        parts = line.strip().split(',')
                        if len(parts) < 8:  # Nếu số phần tử không đủ, bỏ qua
                            continue

                        # Tách tọa độ x, y và văn bản
                        x = list(map(int, parts[:8:2]))  # Tọa độ x
                        y = list(map(int, parts[1:8:2]))  # Tọa độ y
                        transcription = parts[8]  # Văn bản

                        # # Nếu văn bản là '###', bỏ qua
                        # if transcription == '###':
                        #     continue
                        
                        # Tính toán bounding box YOLO
                        x_center, y_center, bbox_width, bbox_height = polygon_to_yolo_bbox(x, y, width_img, height_img)
                        
                        # Nếu bounding box hợp lệ, đánh dấu
                        if bbox_width > 0 and bbox_height > 0:
                            valid_boxes = True
                        
                        # Gán class_id (ví dụ: 0 cho tất cả các lớp trong OCR)
                        class_id = 0
                        
                        # Ghi vào file annotation
                        out_f.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")
                    except Exception as e:
                        print(f"Lỗi xử lý dòng trong file {anno_file}: {line}\nChi tiết lỗi: {e}")
            
            # Chỉ lưu ảnh và file annotation nếu có ít nhất một bounding box hợp lệ
            if valid_boxes:
                img.save(output_image_path)  # Lưu ảnh vào thư mục output
            else:
                os.remove(output_label_file)  # Xóa file label nếu không có bounding box hợp lệ
                print(f"Không có bounding box hợp lệ cho ảnh {base_name}.jpg, file annotation không được lưu.")
                
        except Exception as e:
            print(f"Lỗi xử lý file: {anno_file}\nChi tiết lỗi: {e}")

In [7]:
Icidar2015_process_annotations(icdar2015_test_images_folder, icdar2015_test_labels_folder, icdar2015_test_output_images_folder, icdar2015_test_output_labels_folder)

# Total Dataset test


In [8]:
def Total_process_annotations(train_images_folder, train_labels_folder, train_output_images_folder, train_output_labels_folder):
    # Tạo các thư mục đầu ra nếu chưa tồn tại
    os.makedirs(train_output_images_folder, exist_ok=True)
    os.makedirs(train_output_labels_folder, exist_ok=True)

    # Hàm chuyển đổi tọa độ polygon sang YOLO bounding box
    def polygon_to_yolo_bbox(x, y, width_img, height_img):
        x_min, x_max = min(x), max(x)
        y_min, y_max = min(y), max(y)
        x_center = (x_min + x_max) / 2 / width_img
        y_center = (y_min + y_max) / 2 / height_img
        bbox_width = (x_max - x_min) / width_img
        bbox_height = (y_max - y_min) / height_img
        return x_center, y_center, bbox_width, bbox_height

    # Duyệt qua các file annotation
    annotation_files = glob.glob(os.path.join(train_labels_folder, "*.txt"))

    for anno_file in annotation_files:
        base_name = os.path.basename(anno_file).replace("poly_gt_", "").replace(".txt", "")
        image_path = os.path.join(train_images_folder, f"{base_name}.jpg")
        
        try:
            # Mở ảnh để lấy kích thước
            img = Image.open(image_path)
            width_img, height_img = img.size

            # Lưu ảnh vào thư mục đầu ra
            output_image_path = os.path.join(train_output_images_folder, f"{base_name}.jpg")
            
            # Biến kiểm tra xem có bounding box hợp lệ không
            valid_boxes = False
            
            # Lưu file annotation mới vào thư mục đầu ra
            output_label_file = os.path.join(train_output_labels_folder, f"{base_name}.txt")
            
            with open(anno_file, "r") as f, open(output_label_file, "w") as out_f:
                for line in f:
                    try:
                        # Tách các phần tử trong dòng annotation và bỏ qua các phần không cần thiết
                        line = line.strip()
                        if "x: [[" not in line or "y: [[" not in line:
                            continue

                        # Lấy phần tọa độ x và y
                        x_part = line[line.index("x: [[") + 5: line.index("]], y:")]
                        y_part = line[line.index("y: [[") + 5: line.index("]], ornt:")]

                        # Chuyển đổi tọa độ x và y thành danh sách số
                        x = list(map(int, x_part.split()))
                        y = list(map(int, y_part.split()))

                        # Bỏ qua nếu số điểm không đủ
                        if len(x) < 4 or len(y) < 4:
                            continue
                        
                        # Tính toán bounding box YOLO
                        x_center, y_center, bbox_width, bbox_height = polygon_to_yolo_bbox(x, y, width_img, height_img)
                        
                        # Nếu bounding box hợp lệ, đánh dấu
                        if bbox_width > 0 and bbox_height > 0:
                            valid_boxes = True
                        
                        # Gán class_id (ví dụ: 0 cho tất cả các lớp trong OCR)
                        class_id = 0
                        
                        # Ghi vào file annotation
                        out_f.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")
                    except Exception as e:
                        print(f"Lỗi xử lý dòng trong file {anno_file}: {line}\nChi tiết lỗi: {e}")
            
            # Chỉ lưu ảnh và file annotation nếu có ít nhất một bounding box hợp lệ
            if valid_boxes:
                img.save(output_image_path)  # Lưu ảnh vào thư mục output
            else:
                os.remove(output_label_file)  # Xóa file label nếu không có bounding box hợp lệ
                print(f"Không có bounding box hợp lệ cho ảnh {base_name}.jpg, file annotation không được lưu.")
                
        except Exception as e:
            print(f"Lỗi xử lý file: {anno_file}\nChi tiết lỗi: {e}")

In [9]:
Total_process_annotations(total_test_images_folder, total_test_labels_folder, total_test_output_images_folder, total_test_output_labels_folder)


# BKAI DATASET TEST


In [10]:
import os
import glob
from PIL import Image

def test_process_annotations(train_images_folder, train_labels_folder, train_output_images_folder, train_output_labels_folder):
    # Tạo các thư mục đầu ra nếu chưa tồn tại
    os.makedirs(train_output_images_folder, exist_ok=True)
    os.makedirs(train_output_labels_folder, exist_ok=True)

    # Hàm chuyển đổi tọa độ polygon sang YOLO bounding box
    def polygon_to_yolo_bbox(x, y, width_img, height_img):
        x_min, x_max = min(x), max(x)
        y_min, y_max = min(y), max(y)
        x_center = (x_min + x_max) / 2 / width_img
        y_center = (y_min + y_max) / 2 / height_img
        bbox_width = (x_max - x_min) / width_img
        bbox_height = (y_max - y_min) / height_img
        return x_center, y_center, bbox_width, bbox_height

    # Duyệt qua các file annotation
    annotation_files = glob.glob(os.path.join(train_labels_folder, "*.txt"))

    for anno_file in annotation_files:
        base_name = os.path.basename(anno_file).replace("gt_", "").replace(".txt", "")
        image_path = os.path.join(train_images_folder, f"{base_name}.jpg")
        
        try:
            # Mở ảnh để lấy kích thước
            img = Image.open(image_path)
            width_img, height_img = img.size

            # Lưu ảnh vào thư mục đầu ra
            output_image_path = os.path.join(train_output_images_folder, f"{base_name}.jpg")
            
            # Biến kiểm tra xem có bounding box hợp lệ không
            valid_boxes = False
            
            # Lưu file annotation mới vào thư mục đầu ra
            output_label_file = os.path.join(train_output_labels_folder, f"{base_name}.txt")
            
            with open(anno_file, "r", encoding="utf-8-sig") as f, open(output_label_file, "w") as out_f:
                for line in f:
                    try:
                        # Loại bỏ ký tự BOM nếu có
                        line = line.lstrip('\ufeff')

                        # Tách các phần tử trong dòng annotation
                        parts = line.strip().split(',')
                        if len(parts) < 8:  # Nếu số phần tử không đủ, bỏ qua
                            continue

                        # Tách tọa độ x, y và văn bản
                        x = list(map(int, parts[:8:2]))  # Tọa độ x
                        y = list(map(int, parts[1:8:2]))  # Tọa độ y
                        transcription = parts[8]  # Văn bản

                        # Nếu bounding box không hợp lệ, bỏ qua
                        x_center, y_center, bbox_width, bbox_height = polygon_to_yolo_bbox(x, y, width_img, height_img)
                        if bbox_width > 0 and bbox_height > 0:
                            valid_boxes = True
                        
                        # Gán class_id (ví dụ: 0 cho tất cả các lớp trong OCR)
                        class_id = 0
                        
                        # Ghi vào file annotation
                        out_f.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")
                    except Exception as e:
                        print(f"Lỗi xử lý dòng trong file {anno_file}: {line}\nChi tiết lỗi: {e}")
            
            # Chỉ lưu ảnh và file annotation nếu có ít nhất một bounding box hợp lệ
            if valid_boxes:
                # Chuyển đổi chế độ ảnh nếu cần thiết
                if img.mode == "RGBA" or img.mode == "P":
                    img = img.convert("RGB")
                img.save(output_image_path, "JPEG",quality=95)  # Lưu ảnh vào thư mục output
            else:
                os.remove(output_label_file)  # Xóa file label nếu không có bounding box hợp lệ
                print(f"Không có bounding box hợp lệ cho ảnh {base_name}.jpg, file annotation không được lưu.")
                
        except Exception as e:
            print(f"Lỗi xử lý file: {anno_file}\nChi tiết lỗi: {e}")

In [11]:
test_images_folder = "/kaggle/input/test-bkai/private_test_imgs"
test_labels_folder = "/kaggle/input/test-bkai/groundtruth"
test_output_images_folder = "/kaggle/working/test/images"
test_output_labels_folder = "/kaggle/working/test/labels"

test_process_annotations(test_images_folder, test_labels_folder, test_output_images_folder, test_output_labels_folder)

# Data yaml

In [12]:
# Tạo đường dẫn tới file YAML
dataset_dir = "/kaggle/working"
os.makedirs(dataset_dir, exist_ok=True)
yaml_file = os.path.join(dataset_dir, "data_BKAI.yaml")

# Nội dung file YAML
yaml_content = """
train: /kaggle/input/yolov8x-ocr/train/images
val: /kaggle/working/test/images
nc: 1
names: ['text']  # Cập nhật nếu bạn có nhiều lớp

"""

# Ghi nội dung vào file YAML
with open(yaml_file, "w") as f:
    f.write(yaml_content.strip())

print(f"File 'data_BKAI.yaml' đã được tạo tại: {yaml_file}")

File 'data_BKAI.yaml' đã được tạo tại: /kaggle/working/data_BKAI.yaml


In [13]:
# Tạo đường dẫn tới file YAML
dataset_dir = "/kaggle/working"
os.makedirs(dataset_dir, exist_ok=True)
yaml_file = os.path.join(dataset_dir, "data_Total.yaml")

# Nội dung file YAML
yaml_content = """
train: /kaggle/input/yolov8x-ocr/train/images
val: /kaggle/working/test_total/images
nc: 1
names: ['text']  # Cập nhật nếu bạn có nhiều lớp

"""

# Ghi nội dung vào file YAML
with open(yaml_file, "w") as f:
    f.write(yaml_content.strip())

print(f"File 'data_Total.yaml' đã được tạo tại: {yaml_file}")

File 'data_Total.yaml' đã được tạo tại: /kaggle/working/data_Total.yaml


In [14]:
# Tạo đường dẫn tới file YAML
dataset_dir = "/kaggle/working"
os.makedirs(dataset_dir, exist_ok=True)
yaml_file = os.path.join(dataset_dir, "data_Icdar2015.yaml")

# Nội dung file YAML
yaml_content = """
train: /kaggle/input/yolov8x-ocr/train/images
val: /kaggle/working/test_icdar2015/images
nc: 1
names: ['text']  # Cập nhật nếu bạn có nhiều lớp

"""

# Ghi nội dung vào file YAML
with open(yaml_file, "w") as f:
    f.write(yaml_content.strip())

print(f"File 'data_Icdar2015.yaml' đã được tạo tại: {yaml_file}")

File 'data_Icdar2015.yaml' đã được tạo tại: /kaggle/working/data_Icdar2015.yaml


# Train lai - Step 2 - SGD ver 1



In [15]:
from ultralytics import YOLO
import os
import cv2

# Đường dẫn thư mục chứa các ảnh và mô hình YOLOv8
image_path = '/kaggle/input/test-bkai/private_test_imgs'
model_path = '/kaggle/input/optimizer-sgd/pytorch/default/1/Step 2/Step 2 Vinhx Ver 1/best.pt'

# Load mô hình YOLOv8 từ đường dẫn
model = YOLO(model_path)
results = model.val(data='/kaggle/working/data_BKAI.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 268 layers, 68,124,531 parameters, 0 gradients, 257.4 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 25.4MB/s]
val: Scanning /kaggle/working/test/labels... 300 images, 0 backgrounds, 1 corrupt: 100%|██████████| 300/300 [00:00<00:00, 1191.98it/s]

val: WARNING ⚠️ /kaggle/working/test/images/img_238.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1983]
val: New cache created: /kaggle/working/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:07<00:00,  2.46it/s]


                   all        299        866      0.899      0.857      0.907      0.622
Speed: 0.5ms preprocess, 16.6ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/val
Overall Metrics:
Average Precision: 0.8994
Average Recall: 0.8570
Average F1-Score: 0.8777
mAP50: 0.9071
mAP50-95: 0.6221


In [16]:
results = model.val(data='/kaggle/working/data_Icdar2015.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_icdar2015/labels... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<00:00, 1151.15it/s]

val: New cache created: /kaggle/working/test_icdar2015/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:11<00:00,  2.77it/s]


                   all        500       5230      0.613      0.456      0.487      0.243
Speed: 0.5ms preprocess, 16.4ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/val2
Overall Metrics:
Average Precision: 0.6130
Average Recall: 0.4556
Average F1-Score: 0.5227
mAP50: 0.4870
mAP50-95: 0.2430


In [17]:
results = model.val(data='/kaggle/working/data_Total.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_total/labels... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<00:00, 1154.36it/s]

val: New cache created: /kaggle/working/test_total/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:09<00:00,  2.10it/s]


                   all        300       2540      0.715      0.607      0.647        0.4
Speed: 1.0ms preprocess, 22.2ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs/detect/val3
Overall Metrics:
Average Precision: 0.7146
Average Recall: 0.6067
Average F1-Score: 0.6562
mAP50: 0.6472
mAP50-95: 0.3997


# Train lai - Step 2 - SGD ver 2



In [18]:
from ultralytics import YOLO
import os
import cv2

# Đường dẫn thư mục chứa các ảnh và mô hình YOLOv8
image_path = '/kaggle/input/test-bkai/private_test_imgs'
model_path = '/kaggle/input/optimizer-sgd/pytorch/default/1/Step 2/Step 2 Vinhx Ver 2/best (1).pt'

# Load mô hình YOLOv8 từ đường dẫn
model = YOLO(model_path)
results = model.val(data='/kaggle/working/data_BKAI.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 268 layers, 68,124,531 parameters, 0 gradients, 257.4 GFLOPs


val: Scanning /kaggle/working/test/labels.cache... 300 images, 0 backgrounds, 1 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]

val: WARNING ⚠️ /kaggle/working/test/images/img_238.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1983]



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:06<00:00,  2.99it/s]


                   all        299        866      0.915      0.851      0.915      0.639
Speed: 0.3ms preprocess, 15.9ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/val4
Overall Metrics:
Average Precision: 0.9152
Average Recall: 0.8510
Average F1-Score: 0.8820
mAP50: 0.9151
mAP50-95: 0.6387


In [19]:
results = model.val(data='/kaggle/working/data_Icdar2015.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_icdar2015/labels.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:11<00:00,  2.76it/s]


                   all        500       5230      0.657      0.542      0.589      0.312
Speed: 0.6ms preprocess, 16.0ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs/detect/val5
Overall Metrics:
Average Precision: 0.6571
Average Recall: 0.5419
Average F1-Score: 0.5940
mAP50: 0.5894
mAP50-95: 0.3117


In [20]:
results = model.val(data='/kaggle/working/data_Total.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_total/labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:09<00:00,  2.01it/s]


                   all        300       2540      0.753      0.672       0.74      0.481
Speed: 1.2ms preprocess, 20.5ms inference, 0.0ms loss, 4.2ms postprocess per image
Results saved to runs/detect/val6
Overall Metrics:
Average Precision: 0.7531
Average Recall: 0.6724
Average F1-Score: 0.7105
mAP50: 0.7400
mAP50-95: 0.4812


# Train lai - Step 2 - SGD ver 3


In [21]:
from ultralytics import YOLO
import os
import cv2

# Đường dẫn thư mục chứa các ảnh và mô hình YOLOv8
image_path = '/kaggle/input/test-bkai/private_test_imgs'
model_path = '/kaggle/input/optimizer-sgd/pytorch/default/1/Step 2/Step 2 Vinhx Ver 3/best.pt'

# Load mô hình YOLOv8 từ đường dẫn
model = YOLO(model_path)
results = model.val(data='/kaggle/working/data_BKAI.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 268 layers, 68,124,531 parameters, 0 gradients, 257.4 GFLOPs


val: Scanning /kaggle/working/test/labels.cache... 300 images, 0 backgrounds, 1 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]

val: WARNING ⚠️ /kaggle/working/test/images/img_238.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1983]



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:06<00:00,  2.98it/s]


                   all        299        866      0.924      0.843      0.915      0.631
Speed: 0.6ms preprocess, 16.0ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/val7
Overall Metrics:
Average Precision: 0.9240
Average Recall: 0.8429
Average F1-Score: 0.8816
mAP50: 0.9151
mAP50-95: 0.6314


In [22]:
results = model.val(data='/kaggle/working/data_Icdar2015.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_icdar2015/labels.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:11<00:00,  2.73it/s]


                   all        500       5230      0.663      0.532      0.583      0.307
Speed: 0.6ms preprocess, 15.8ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/val8
Overall Metrics:
Average Precision: 0.6627
Average Recall: 0.5320
Average F1-Score: 0.5902
mAP50: 0.5835
mAP50-95: 0.3070


In [23]:
results = model.val(data='/kaggle/working/data_Total.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_total/labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:09<00:00,  1.99it/s]


                   all        300       2540      0.743      0.681      0.736      0.474
Speed: 0.8ms preprocess, 21.9ms inference, 0.0ms loss, 3.4ms postprocess per image
Results saved to runs/detect/val9
Overall Metrics:
Average Precision: 0.7427
Average Recall: 0.6807
Average F1-Score: 0.7104
mAP50: 0.7361
mAP50-95: 0.4742


# Train lai - Step 2 - AdamW ver 1


In [24]:
from ultralytics import YOLO
import os
import cv2

# Đường dẫn thư mục chứa các ảnh và mô hình YOLOv8
image_path = '/kaggle/input/test-bkai/private_test_imgs'
model_path = '/kaggle/input/optimizer-adamw/pytorch/default/1/Step 2/Step 2 UIT ver 1/best (1).pt'

# Load mô hình YOLOv8 từ đường dẫn
model = YOLO(model_path)
results = model.val(data='/kaggle/working/data_BKAI.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 268 layers, 68,124,531 parameters, 0 gradients, 257.4 GFLOPs


val: Scanning /kaggle/working/test/labels.cache... 300 images, 0 backgrounds, 1 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]

val: WARNING ⚠️ /kaggle/working/test/images/img_238.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1983]



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:06<00:00,  3.05it/s]


                   all        299        866      0.884        0.8      0.875      0.582
Speed: 0.3ms preprocess, 15.5ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/val10
Overall Metrics:
Average Precision: 0.8839
Average Recall: 0.8000
Average F1-Score: 0.8399
mAP50: 0.8748
mAP50-95: 0.5819


In [25]:
results = model.val(data='/kaggle/working/data_Icdar2015.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_icdar2015/labels.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:12<00:00,  2.66it/s]


                   all        500       5230      0.666       0.51      0.559      0.291
Speed: 0.5ms preprocess, 16.1ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to runs/detect/val11
Overall Metrics:
Average Precision: 0.6656
Average Recall: 0.5101
Average F1-Score: 0.5776
mAP50: 0.5591
mAP50-95: 0.2915


In [26]:
results = model.val(data='/kaggle/working/data_Total.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_total/labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:09<00:00,  2.06it/s]


                   all        300       2540      0.751      0.652      0.711       0.45
Speed: 1.0ms preprocess, 21.1ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/val12
Overall Metrics:
Average Precision: 0.7508
Average Recall: 0.6516
Average F1-Score: 0.6977
mAP50: 0.7111
mAP50-95: 0.4502


# Train lai - Step 2 - AdamW ver 2


In [27]:
from ultralytics import YOLO
import os
import cv2

# Đường dẫn thư mục chứa các ảnh và mô hình YOLOv8
image_path = '/kaggle/input/test-bkai/private_test_imgs'
model_path = '/kaggle/input/optimizer-adamw/pytorch/default/1/Step 2/Step 2 UIT Ver 2/best.pt'

# Load mô hình YOLOv8 từ đường dẫn
model = YOLO(model_path)
results = model.val(data='/kaggle/working/data_BKAI.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 268 layers, 68,124,531 parameters, 0 gradients, 257.4 GFLOPs


val: Scanning /kaggle/working/test/labels.cache... 300 images, 0 backgrounds, 1 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]

val: WARNING ⚠️ /kaggle/working/test/images/img_238.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1983]



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:06<00:00,  3.04it/s]


                   all        299        866      0.883      0.738       0.83      0.541
Speed: 0.6ms preprocess, 16.2ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/val13
Overall Metrics:
Average Precision: 0.8825
Average Recall: 0.7375
Average F1-Score: 0.8035
mAP50: 0.8304
mAP50-95: 0.5410


In [28]:
results = model.val(data='/kaggle/working/data_Icdar2015.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_icdar2015/labels.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:11<00:00,  2.75it/s]


                   all        500       5230      0.653      0.508      0.546      0.282
Speed: 0.4ms preprocess, 15.5ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to runs/detect/val14
Overall Metrics:
Average Precision: 0.6535
Average Recall: 0.5077
Average F1-Score: 0.5714
mAP50: 0.5464
mAP50-95: 0.2820


In [29]:
results = model.val(data='/kaggle/working/data_Total.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_total/labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:09<00:00,  2.06it/s]


                   all        300       2540       0.66      0.596      0.627      0.376
Speed: 0.7ms preprocess, 21.4ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to runs/detect/val15
Overall Metrics:
Average Precision: 0.6596
Average Recall: 0.5964
Average F1-Score: 0.6264
mAP50: 0.6269
mAP50-95: 0.3764


# Train lai - Step 2 - AdamW ver 3


In [30]:
from ultralytics import YOLO
import os
import cv2

# Đường dẫn thư mục chứa các ảnh và mô hình YOLOv8
image_path = '/kaggle/input/test-bkai/private_test_imgs'
model_path = '/kaggle/input/optimizer-adamw/pytorch/default/1/Step 2/Step 2 UIT Ver 3/best.pt'

# Load mô hình YOLOv8 từ đường dẫn
model = YOLO(model_path)
results = model.val(data='/kaggle/working/data_BKAI.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 268 layers, 68,124,531 parameters, 0 gradients, 257.4 GFLOPs


val: Scanning /kaggle/working/test/labels.cache... 300 images, 0 backgrounds, 1 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]

val: WARNING ⚠️ /kaggle/working/test/images/img_238.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1983]



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:06<00:00,  3.10it/s]


                   all        299        866      0.894      0.809      0.888      0.592
Speed: 0.5ms preprocess, 16.0ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/val16
Overall Metrics:
Average Precision: 0.8941
Average Recall: 0.8091
Average F1-Score: 0.8495
mAP50: 0.8880
mAP50-95: 0.5919


In [31]:
results = model.val(data='/kaggle/working/data_Icdar2015.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_icdar2015/labels.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:11<00:00,  2.70it/s]


                   all        500       5230      0.651      0.537      0.576      0.305
Speed: 0.5ms preprocess, 16.0ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to runs/detect/val17
Overall Metrics:
Average Precision: 0.6511
Average Recall: 0.5373
Average F1-Score: 0.5887
mAP50: 0.5761
mAP50-95: 0.3054


In [32]:
results = model.val(data='/kaggle/working/data_Total.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_total/labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:09<00:00,  2.07it/s]


                   all        300       2540       0.74      0.643      0.705      0.444
Speed: 0.8ms preprocess, 21.2ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to runs/detect/val18
Overall Metrics:
Average Precision: 0.7402
Average Recall: 0.6425
Average F1-Score: 0.6879
mAP50: 0.7046
mAP50-95: 0.4442


# Train lai - Step 2 - AdamW ver 4


In [33]:
from ultralytics import YOLO
import os
import cv2

# Đường dẫn thư mục chứa các ảnh và mô hình YOLOv8
image_path = '/kaggle/input/test-bkai/private_test_imgs'
model_path = '/kaggle/input/optimizer-adamw/pytorch/default/1/Step 2/Step 2 UIT Ver 4/best.pt'

# Load mô hình YOLOv8 từ đường dẫn
model = YOLO(model_path)
results = model.val(data='/kaggle/working/data_BKAI.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 268 layers, 68,124,531 parameters, 0 gradients, 257.4 GFLOPs


val: Scanning /kaggle/working/test/labels.cache... 300 images, 0 backgrounds, 1 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]

val: WARNING ⚠️ /kaggle/working/test/images/img_238.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1983]



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:06<00:00,  3.03it/s]


                   all        299        866      0.898      0.813      0.887      0.589
Speed: 0.3ms preprocess, 15.8ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/val19
Overall Metrics:
Average Precision: 0.8979
Average Recall: 0.8129
Average F1-Score: 0.8533
mAP50: 0.8867
mAP50-95: 0.5887


In [34]:
results = model.val(data='/kaggle/working/data_Icdar2015.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_icdar2015/labels.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:11<00:00,  2.72it/s]


                   all        500       5230      0.665      0.523      0.572      0.301
Speed: 0.6ms preprocess, 17.0ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/val20
Overall Metrics:
Average Precision: 0.6649
Average Recall: 0.5231
Average F1-Score: 0.5856
mAP50: 0.5721
mAP50-95: 0.3012


In [35]:
results = model.val(data='/kaggle/working/data_Total.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_total/labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:09<00:00,  2.01it/s]


                   all        300       2540      0.739      0.634      0.702      0.443
Speed: 0.8ms preprocess, 20.2ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/val21
Overall Metrics:
Average Precision: 0.7387
Average Recall: 0.6339
Average F1-Score: 0.6823
mAP50: 0.7022
mAP50-95: 0.4433
